In [17]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

[autoreload of fnsa.lexicon failed: Traceback (most recent call last):
  File "/Users/rflagg/anaconda3/envs/fsa/lib/python3.5/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: cannot import name 'EXTRACTED_ENTITY_LEX'
]


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
from fnsa.lexicon import Lexicon, prepare
from fnsa.util import *
import spacy

In [19]:
nlp = spacy.load('en_core_web_sm')
sentences = load_sentences()

In [54]:
lexicon = Lexicon(nlp)

In [15]:
for i, sentence in enumerate(sentences[:1]):
    doc = lexicon(sentence)
    show(doc, index=(i+1), include_text=True)
    print("-----")

[('fe', '+', 4, 5, -1, False), ('if', 'rev', 8, 9, -1, False), ('if', 'mod', 5, 6, -1, False), ('if', 'mod', 6, 7, -1, False), ('lm', '+', 5, 6, -1, False), ('lm', '-', 8, 9, -1, False), ('xe', 'org', 1, 2, -1, True), ('xe', 'org', 4, 5, -1, True)]
1. The Group's operative EBIT will probably be negative.
   The [Group/xe:org] 's operative [EBIT/fe:+] [will/if:mod] [probably/if:mod] be [negative/if:rev] .
-----


In [16]:
for lex, category, start, end, index, acceptable in doc.user_data['entries']:
    print("%s --> %s/%s" % (doc[start:end], lex, category))

Group --> xe/org
probably --> if/mod
EBIT --> fe/+
negative --> if/rev
will --> if/mod


In [ ]:
doc = self.nlp(text)
prepare(doc, self.lexicons)
for e in doc.ents: 
    store_match(doc, EXTRACTED_ENTITY_LEX, e.label_.lower()[:3], e.start, e.end, -1, e.label_ in ACCEPTABLE_TYPES)
self.match(doc)
self.merge(doc)




In [55]:
import json
sentence = 'Earnings per share ( EPS ) amounted to EUR0 .98 , up from the loss of EUR0 .02 .'
doc = nlp(sentence)
print(sentence)
prepare(doc, lexicon.lexicons)
print(doc)
#print(json.dumps(doc.user_data, indent=4, sort_keys=True))
#print(doc.user_data)
lexicon.match(doc)
print(doc)
lexicon.merge(doc)
#print(doc)


#print(json.dumps(doc.user_data, indent=4, sort_keys=True))
#print(doc.user_data)

#annotations = annotate(doc)
#print(" ".join(annotations))
#for token in doc: print("[%s/%s:%s]" % (token.text, token._.lex, token._.category))

Earnings per share ( EPS ) amounted to EUR0 .98 , up from the loss of EUR0 .02 .
Earnings per share ( EPS ) amounted to EUR0 .98 , up from the loss of EUR0 .02 .
Earnings per share ( EPS ) amounted to EUR0 .98 , up from the loss of EUR0 .02 .


In [56]:
sentence = 'Earnings per share ( EPS ) amounted to EUR0 .98 , up from the loss of EUR0 .02 .'
doc = lexicon(sentence)
show(doc, include_text=True)

Earnings per share ( EPS ) amounted to EUR0 .98 , up from the loss of EUR0 .02 .
 [Earnings per share/fe:+] ( [EPS/fe:+] ) amounted to [EUR0/xe:org] .98 , [up/dr:+] from the [loss/fe:-] of [EUR0/xe:org] .02 .


In [26]:
??show

In [30]:
doc.user_data

{('._.', 'lex', 70, None): 'xe',
 'en2scope': {},
 ('._.', 'lex', 0, None): 'fe',
 ('._.', 'category', 50, None): '+',
 ('._.', 'lex', 62, None): 'fe',
 ('._.', 'category', 0, None): '+',
 ('._.', 'category', 70, None): 'org',
 'fi_matched': False,
 'fe_matched': True,
 ('._.', 'lex', 39, None): 'xe',
 ('._.', 'category', 62, None): '-',
 'entries': {('dr', '+', 11, 12, -1, False),
  ('fe', '+', 0, 3, -1, False),
  ('fe', '+', 4, 5, -1, False),
  ('fe', '-', 14, 15, -1, False),
  ('xe', 'org', 8, 9, -1, True),
  ('xe', 'org', 16, 17, -1, True)},
 'lm_matched': True,
 'if_matched': False,
 ('._.', 'category', 39, None): 'org',
 'dr_matched': True,
 ('._.', 'lex', 21, None): 'fe',
 ('._.', 'lex', 50, None): 'dr',
 ('._.', 'category', 21, None): '+'}